In [63]:
import pandas as pd
import sqlalchemy as db
import psycopg2 
import random

# Connect to the database via SQLalchemy
engine = db.create_engine('postgresql://{user}:{user_pass}@{host}/{dataname1}')

# Connect via psycopg2
conn = psycopg2.connect('dbname=dataname1 user={user} password={user_pass}')
cur = conn.cursor()

In [64]:
# Create a test data frame
df = pd.DataFrame(columns = ["id", "phone_request_needed", "phone_request_made"], index = range(0, 2000))
for i in range(0, 2000): 
    df.iloc[i] = [i, random.randint(0, 1), random.randint(0, 1)]
# Put into Postgresql
df.to_sql('test_requests', engine)

In [66]:
# Select a limited number of rows
working_table = pd.read_sql('SELECT * FROM test_requests '
                            'WHERE phone_request_made = 0 AND phone_request_needed = 1 LIMIT 100', 
                            engine)

In [67]:
working_table

,index,id,phone_request_needed,phone_request_made
0,1,1,1,0
1,7,7,1,0
2,9,9,1,0
3,13,13,1,0
4,14,14,1,0
...,...,...,...,...
95,385,385,1,0
96,388,388,1,0
97,393,393,1,0
98,401,401,1,0


In [68]:
assert (sum(working_table['phone_request_made'] == 1) == 0)

In [69]:
# Update rows 
working_table['phone_request_made'] = 1

In [71]:
# Update the original table
cur.execute("""CREATE TEMP TABLE updated_rows(id INTEGER, phone_request_made INTEGER) ON COMMIT DROP""")
rows = zip(working_table.id, working_table.phone_request_made)
cur.executemany("""INSERT INTO updated_rows (id, phone_request_made) VALUES(%s, %s)""", rows)
cur.execute("""
    UPDATE test_requests
    SET phone_request_made = updated_rows.phone_request_made
    FROM updated_rows
    WHERE updated_rows.id = test_requests.id;
    """)
conn.commit()

In [73]:
# See the changes
pd.read_sql('SELECT * FROM test_requests', engine)

,index,id,phone_request_needed,phone_request_made
0,0,0,0,0
1,2,2,0,0
2,3,3,1,1
3,4,4,0,1
4,5,5,0,0
...,...,...,...,...
1995,385,385,1,1
1996,388,388,1,1
1997,393,393,1,1
1998,401,401,1,1


In [74]:
# Close connection
engine.dispose()
conn.close()

In [4]:
import pandas as pd
import random
df = pd.DataFrame(columns = ["id", "phone_request_needed", "phone_request_made"], index = range(0, 10))
for i in range(0, 10): 
    df.iloc[i] = [i, random.randint(0, 1), random.randint(0, 1)]
df

,id,phone_request_needed,phone_request_made
0,0,0,1
1,1,0,1
2,2,1,1
3,3,1,0
4,4,1,0
5,5,0,0
6,6,1,0
7,7,0,1
8,8,0,0
9,9,1,0


In [37]:
def test_func(row):
    temp = [{'id': row['id'], 'a': 4, 'b': 2},
            {'id': row['id'], 'a': 1, 'b': 3}
           ]
    return pd.DataFrame(temp)

In [40]:
list(df.apply(test_func, axis = 1))

,id,a,b
0,1,4,2
1,1,1,3


In [ ]:
api_key = '***REMOVED***'


In [3]:
####### Checking getting addresses from restaurants_requests table

import logging # Facilitates logging 
import psycopg2 # PostgreSQL interaction
import sqlalchemy as db# SQL interaction
import pandas as pd # Data tables

get_addresses_statement = """ 
SELECT 
    * 
FROM 
    restaurants_requests
WHERE 
    address_request_status = 'needed' 
LIMIT {n_records}
"""

create_temp_update_table_statement = """ 
CREATE TEMPORARY TABLE updated_rows (row_id INTEGER, address_request_status VARCHAR) 
ON COMMIT DROP
"""

insert_temp_update_table_statement = """ 
INSERT INTO updated_rows (row_id, address_request_status) 
VALUES(%s, %s)
"""

update_matching_table_statement = """ 
UPDATE restaurants_requests
SET 
    address_request_status = updated_rows.address_request_status
FROM updated_rows
WHERE 
    updated_rows.row_id = restaurants_requests.row_id;
"""

In [4]:
# Connect to the database via SQLalchemy
engine = db.create_engine('postgresql://{user}:{user_pass}@{host}/{dataname1}')
l_get_addresses_statement = get_addresses_statement.format(n_records = 25)

In [5]:
requests_table = pd.read_sql(l_get_addresses_statement, engine)

In [6]:
requests_table

,row_id,sname_place_id,r_phone_number,r_location_name,r_zip_code,r_street_address,r_city,r_state,y_id,phone_request_status,address_request_status,phone_error_message,address_error_message
0,224158,sg:7e69872209114effb165de63ea2637c4,None,Bonchon,55425,372 south avenue,bloomington,mn,NKdm97TffSJWRg9z7SG4Yg,unavail,needed,None,None
1,249862,sg:c073c0925bb642158561e3f8fadafecd,None,Zaxby's,30241,129 commerce avenue,lagrange,ga,aO_GM0Zfm8a7pJ2BzJq0bA,unavail,needed,None,None
2,250036,sg:7c349c0a85a64b36a3fe3ff5ef9cb86c,None,Zaxby's,35125,280 vaughan lane,pell city,al,yFoowyYPRrT4Th-hr6ioAA,unavail,needed,None,None
3,261598,sg:eefa90d941ee4087a5fff4686dba4c8a,None,Lunch Box,70507,4302 moss street,lafayette,la,MbMv2s38wHivrjLgDO29oA,unavail,needed,None,None
4,275248,sg:98a5ccd2df5049398c45bb5315aef0f3,None,Subway,89169,3900 paradise road,las vegas,nv,D8a2usm9b-j89W9IrJ_Evw,unavail,needed,None,None
5,287806,sg:b645f80dc0614f16a3d8c30646ed1b1f,None,Le Cellier Steakhouse,32830,1510 n avenue,orlando,fl,None,unavail,needed,None,None
6,287953,sg:13afa391dae6450fb09893f10adcd9d7,None,Subway,77590,2920 palmer highway,texas city,tx,None,unavail,needed,None,None
7,288103,sg:13d19d27822646ec9aa33dd51950757a,None,Wendy's,77493,1484 katy fort bend road,katy,tx,None,unavail,needed,None,None
8,288153,sg:b6477536fea142d3aefe13e55e13b59b,None,J Walken Chicken,35179,275 county road 1242,west point,al,None,unavail,needed,None,None
9,288845,sg:b621daf0d0e8439c84bb1fe7158cd328,None,Meadow Grill,95389,curry village,yosemite national park,ca,None,unavail,needed,None,None


In [7]:
engine.dispose()